In [1]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [2]:
async def get_expiry(f_today):

    if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
        f_expiry = dt.date(2024, 1, 25)
    elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
        f_expiry = dt.date(2024, 1, 31)
    elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
        f_expiry = dt.date(2024, 2, 29)
    elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
        f_expiry = dt.date(2024, 2, 27)
    elif f_today < dt.date(2023, 9, 1):
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
    elif f_today >= dt.date(2023, 9, 1):
        if f_today.day < 24:
            days_to_wednesday = (2 - f_today.weekday()) % 7
            nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
            f_expiry = nearest_wednesday
            if nse.valid_days(
                start_date=nearest_wednesday, end_date=nearest_wednesday
            ).empty:
                f_expiry = nearest_wednesday - dt.timedelta(days=1)
        else:
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(
                start_date=nearest_thursday, end_date=nearest_thursday
            ).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_finnifty(f_today):

    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_expiry_midcpnifty(f_today):

    days_to_thursday = (0 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [31]:
bnf_pandas = pd.read_csv("../data/fin.csv")

In [32]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [33]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [34]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [35]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

In [36]:
def generate_signals(df, signal_ma, candles_in_num):
    df["c"] = pd.to_numeric(df["c"], errors="coerce")
    df["Signal MA"] = df["c"].rolling(window=signal_ma).mean()
    # df["Trailing MA"] = df["c"].rolling(window=trailing_ma).mean()

    df["Sell Signal"] = 0

    # Generate signals using boolean masking
    sell_signal_mask = df["Signal MA"] <= df["Signal MA"].shift(1)

    for i in range(1, candles_in_num):
        sell_signal_mask &= df["Signal MA"].shift(i) <= df["Signal MA"].shift(i + 1)

    df.loc[sell_signal_mask, "Sell Signal"] = 1

    return df

In [37]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].head())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

181911   2017-01-02 09:15:00
181912   2017-01-02 09:16:00
181913   2017-01-02 09:17:00
181914   2017-01-02 09:18:00
181915   2017-01-02 09:19:00
Name: datetime, dtype: datetime64[ns]


In [38]:
# GLOBAL VARIABLES

# INSTRUMENT = "BANKNIFTY"
# INDEX = "bnf"

# INSTRUMENT = "MIDCPNIFTY"
# INDEX = "midcpnifty"

INSTRUMENT = "FINNIFTY"
INDEX = "finnifty"

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 10

In [39]:
data = data.reset_index(drop=True)
data.set_index("datetime", inplace=True)
data.tail()

open       high        low      close  volume
datetime                                                               
2024-06-07 15:25:00 22162.5000 22166.0500 22157.2500 22163.7000       0
2024-06-07 15:26:00 22163.8000 22165.4000 22158.3500 22163.5000       0
2024-06-07 15:27:00 22165.2500 22172.2000 22160.2000 22169.1500       0
2024-06-07 15:28:00 22169.8500 22171.9500 22164.6500 22168.3000       0
2024-06-07 15:29:00 22170.6000 22171.9000 22162.0000 22166.8000       0

In [40]:
async def trade_option_selling(df, index, strike, asset_class, expiry, signal_ma, no_of_candles, tf, sl):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    eod_exit_flag = False
    remark = ""
    entry_time = None
    no_more_trades = False

    df['datetime'] = pd.to_datetime(df['datetime'])
    # print(df.to_string())

    for i in range(0, len(df)):
        # print(df.iloc[i]['datetime'])
        if not no_more_trades:
            
            points = 0
            current_candle_open = df.iloc[i]["o"]
            current_candle_high = df.iloc[i]["h"]
            current_candle_low = df.iloc[i]["l"]
            current_candle_close = df.iloc[i]["c"]
            subset_df = df[max(0, i-signal_ma):i+1]

            if not in_trade:
                    
                if df.iloc[i]["Sell Signal"] == 1:
                    if not already_signal_exists:
                        # Fresh Sell Signal
    
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # print("Fresh Sell Signal")
                        signal_entry_price = current_candle_low
                        signal_initial_sl = current_candle_low * sl
                        # signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i]["datetime"]
                        already_signal_exists = True
                        # print(signal_creation_time)
    
                    else:
                        if current_candle_low > signal_entry_price:
                            # Better Candle
                            # print(df[df['h'] == max(subset_df['h'])].to_string())
                            # print("Better Candle")
                            signal_entry_price = current_candle_low
                            signal_initial_sl = current_candle_low * sl
                            # signal_initial_sl = max(subset_df['h'])
                            signal_creation_time = df.iloc[i]["datetime"]
                            # print(signal_entry_price, signal_initial_sl)
                            # print(signal_creation_time)
    
                        elif current_candle_low <= signal_entry_price:
    
                            if (
                                current_candle_open < signal_entry_price
                                and df.iloc[i]["datetime"].date()
                                > df.iloc[i - 1]["datetime"].date()
                            ):
                                # Gap Down Condition, Skip Entry
    
                                # print("Gap Down Open, Skip Trade")
                                already_signal_exists = False
                                signal_entry_price = 100000
                                signal_initial_sl = 0
    
                            else:
                                # Entry Triggered
        
                                # print("Entry Triggered")
                                in_trade = True
                                entry_time = df.iloc[i]["datetime"]
                                entry_price = signal_entry_price
                                # print(entry_price, signal_initial_sl)
                                points = 0
    
                else:
                    # Signal Does Not Exist In this Candle
                    if (df.iloc[i - 1]["Sell Signal"] == 1):
                        # print(df[df['h'] == max(subset_df['h'])].to_string())
                        # Condition to Enter the trade even if current candle is not Signal Candle
                        signal_entry_price = df.iloc[i - 1]["l"]
                        signal_initial_sl = df.iloc[i - 1]["l"] * sl
                        # signal_initial_sl = max(subset_df['h'])
                        signal_creation_time = df.iloc[i - 1]["datetime"]
                        already_signal_exists = True
                        # print(signal_entry_price, signal_initial_sl)
                        # print(signal_creation_time)
        
                        if current_candle_low <= signal_entry_price and df.iloc[i]['datetime'] != df.iloc[0]['datetime']:
                            # Entry Triggered
    
                            # print("Entry Triggered !")
                            in_trade = True
                            entry_time = df.iloc[i]["datetime"]
                            entry_price = signal_entry_price
                            points = 0
                            # print(entry_price, signal_initial_sl)
                        else:
                            # Discard Existing Signal
                            
                            # print('Signal Discarded')
                            already_signal_exists = False
                            signal_entry_price = 100000
                            signal_initial_sl = 0
        
            if in_trade:
                # print(df.iloc[i]['datetime'])
                # print('IN TRADE')
                trade_entry_price = signal_entry_price
                trade_initial_sl = signal_initial_sl
                trade_target = 1

                # print(trade_entry_price, trade_initial_sl)

                if trade_entry_price < trade_target:
                    in_trade = False
                    points = 0
                    # continue
                else:
                    if current_candle_open > trade_initial_sl:
                        if (
                            df.iloc[i]["datetime"].date() == entry_time.date()
                            and df.iloc[i]["datetime"].time() == entry_time.time()
                        ):
                            if current_candle_close >= trade_initial_sl:
                                # print('Initial SL Hit')
                                in_trade = False
                                points = -1 * (trade_initial_sl - trade_entry_price)
                                exit_price = trade_initial_sl
                                exit_time = df.iloc[i]["datetime"]
                                remark = "Initial SL hit"
                                max_price = df[:i+1]['h'].max()
                                min_price = df[:i+1]['l'].min()
                                # print(points)
        
                        else:
                            # Gap Open Outside ISL
        
                            # print(df.iloc[i])
                            # print('Gap Open Outside ISL')
                            in_trade = False
                            points = trade_entry_price - current_candle_close
                            exit_price = current_candle_close
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Gap Outside ISL"
                            max_price = df[:i+1]['h'].max()
                            min_price = df[:i+1]['l'].min()
                            # print(points)
        
                    elif current_candle_high >= trade_initial_sl:
                        # Initial SL Hit
        
                        # print('Initial SL Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_initial_sl - trade_entry_price)
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL Hit"
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print(points)
        
                    elif current_candle_low <= trade_target:
                        # Target Hit
        
                        # print('Target Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (trade_target - trade_entry_price)
                        exit_price = trade_target
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Target Hit"
                        no_more_trades = True
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print('NO MORE TRADES')
                        # print(points)
        
                    elif (df.iloc[i]['datetime'].time() >= dt.time(15, 15)) and (df.iloc[i]['datetime'].date() == expiry):
                        # Expiry Exit
        
                        # print('Expiry Exit Condition Hit')
                        # print(current_candle_high, trade_initial_sl)
                        in_trade = False
                        points = -1 * (current_candle_close - trade_entry_price)
                        exit_price = current_candle_close
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Expiry Exit"
                        no_more_trades = True
                        max_price = df[:i+1]['h'].max()
                        min_price = df[:i+1]['l'].min()
                        # print('NO MORE TRADES')
                        # print(points)
                        
                    if points:
                        # print(entry_time)
                        if (entry_time.date() == expiry) and (entry_time.time() >= dt.time(14, 0)):
                            # print('IF CONDITION Triggered in if points:')
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                            # continue
                        
                        else:
                            # print('ELSE CONDITION Triggered in if points:')
                            # qty = int(round(portfolio_value * 5 / entry_price / 15)) * 15
                            qty = int(round(PORTFOLIO_VALUE * INDEX_LEV / strike / 15)) * 15
                            slippage = 0.01 * (entry_price + exit_price)
                            # slippage = 10
                            final_points = points - slippage
                            # final_points = points
                            trade = {
                                "Index": index,
                                "Strike": strike,
                                "Option Type": asset_class,
                                "Expiry": expiry,
                                "Signal Generated At": signal_creation_time,
                                "Trade Type": "SELL",
                                "Entry Date": entry_time.date(),
                                "Entry Time": entry_time.time(),
                                "Entry Price": entry_price,
                                "Initial SL": trade_initial_sl,
                                "Target": trade_target,
                                "Exit Time": exit_time,
                                "Exit Price": exit_price,
                                "Points Captured": points,
                                "Slippages": slippage,
                                "After Costs": final_points,
                                "PnL": final_points * qty,
                                "Remarks": remark,
                                "Qty": qty,
                                # "Leverage": "5x",
                                "ROI%": (final_points * qty / PORTFOLIO_VALUE) * 100,
                                "Max Price": max_price,
                                "Min Price": min_price,
                                "Trade Year": entry_time.year,
                                "Trade Month": entry_time.month,
                                "Variation": f"{signal_ma}, {no_of_candles}, {tf}, {sl}% SL",
                            }
                            # print(trade)
                            trade_book.append(trade)
                            # print("Appended : \n", trade)
                            points = 0
                            in_trade = False
                            already_signal_exists = False
                            remark = ""
                            is_trailing_active = False
                            entry_time = None
                            # no_more_trades = False
                    
        # if no_more_trades:
    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [41]:
# async def convert_to_weekly(df):

#     # FOR BNF 

#     before_df = df[df.index.date < dt.date(2023, 8, 31)]
#     after_df = df[df.index.date >= dt.date(2023, 8, 31)]

#     before_df_resampled = before_df.resample(
#         "W-THU", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     after_df_resampled = after_df.resample(
#         "W-WED", label="right", closed="right"
#     ).agg(
#         {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
#     )

#     weekly_data = pd.concat([before_df_resampled, after_df_resampled])

#     # Reset index if you want to make the DateTime a column again
#     weekly_data = weekly_data.reset_index()

#     return weekly_data



async def convert_to_weekly(df):

    # FOR FNF, NIFTY, MIDCP

    df_resampled = df.resample(
        "W-TUE", label="right", closed="right"
    ).agg(
        {"open": "first", "high": "max", "low": "min", "close": "last", "volume": "sum"}
    )

    weekly_data = pd.DataFrame(df_resampled)
    weekly_data = weekly_data.reset_index()

    return weekly_data

weekly_data = await convert_to_weekly(data)

In [42]:
weekly_data["datetime"] = pd.to_datetime(weekly_data["datetime"])
weekly_data["Week Start"] = weekly_data["datetime"].dt.date - dt.timedelta(days=6)
weekly_data["Expiry"] = weekly_data["datetime"].dt.date

for i in range(0, len(weekly_data)):
    current_expiry = weekly_data["Expiry"].iloc[i]
    if nse.valid_days(start_date=current_expiry, end_date=current_expiry).empty:
        weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
print(weekly_data.to_string())
# await trade(weekly_data)

      datetime       open       high        low      close  volume  Week Start      Expiry
0   2017-01-03  7442.4000  7442.4000  7250.0000  7317.0500       0  2016-12-28  2017-01-03
1   2017-01-10  7325.1000  7416.6000  7257.2000  7409.9000       0  2017-01-04  2017-01-10
2   2017-01-17  7444.6000  7679.6500  7429.0000  7614.3500       0  2017-01-11  2017-01-17
3   2017-01-24  7627.6500  7702.7500  7480.7000  7634.2000       0  2017-01-18  2017-01-24
4   2017-01-31  7658.6000  8006.7500  7658.6000  7906.7000       0  2017-01-25  2017-01-31
5   2017-02-07  7920.9500  8244.9500  7917.6500  8196.8000       0  2017-02-01  2017-02-07
6   2017-02-14  8209.8500  8247.3500  8092.8000  8163.2000       0  2017-02-08  2017-02-14
7   2017-02-21  8171.1500  8460.8000  8116.6000  8399.1000       0  2017-02-15  2017-02-21
8   2017-02-28  8429.8500  8435.5000  8272.8500  8292.7000       0  2017-02-22  2017-02-28
9   2017-03-07  8317.3500  8428.4500  8221.2000  8280.9000       0  2017-03-01  2017-03-07

/tmp/ipykernel_2852/2745470051.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  weekly_data["Expiry"].iloc[i] = current_expiry - dt.timedelta(days=1)
/tmp/ipykernel_2852/2745470051.py:8: SettingWithCopyWarning: 
A value is trying to be se

In [45]:
async def trade(df, signal_ma, no_of_candles, tf, sl):

    combined_trades_ce = pd.DataFrame()
    combined_trades_pe = pd.DataFrame()
    total_trades = pd.DataFrame()

    # print(df.tail().to_string())
    for i in range(300, len(df)):
    # for i in range(0, len(df)):
        current_date = df["Week Start"].iloc[i]
        current_expiry = df["Expiry"].iloc[i]
        # print(i, current_date)
        spot_open = df["open"].iloc[i]
        spot_atm = int(round(spot_open / 50) * 50)
        # print(spot_open, spot_atm)

        # current_expiry = await get_expiry(current_expiry - dt.timedelta(days=1))
        if current_expiry == dt.date(2024, 4, 17):
            current_expiry = dt.date(2024, 4, 16)
        if current_expiry == dt.date(2024, 5, 1):
            current_expiry = dt.date(2024, 4, 30)

        # print(current_expiry)

        ce_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="C",
        )

        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = ce_df.select(["datetime", "o", "h", "l", "c"])
            ce_df = resample(ce_df, tf)
            ce_df_pandas = ce_df.to_pandas()
            ce_df_pandas = ce_df_pandas[~(ce_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
            ce_df = generate_signals(ce_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'CE DF :\n')
            # print(ce_df.to_string())

        else:
            continue

        pe_df = await fetch_data(
            index=INDEX,
            start_date=current_date,
            end_date=current_expiry,
            start_time=dt.time(9, 15),
            end_time=dt.time(15, 30),
            expiry=current_expiry,
            strike=spot_atm,
            asset_class="P",
        )

        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = pe_df.select(["datetime", "o", "h", "l", "c"])
            pe_df = resample(pe_df, tf)
            pe_df_pandas = pe_df.to_pandas()
            pe_df_pandas = pe_df_pandas[~(pe_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
            pe_df = generate_signals(pe_df_pandas, signal_ma, no_of_candles)
            # print(spot_atm, 'PE DF :\n')
            # print(pe_df.to_string())

        # print('Executing CE Trades')
        ce_trades = await trade_option_selling(
            df=ce_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="C",
            expiry=current_expiry,
            signal_ma=signal_ma,
            no_of_candles=no_of_candles,
            tf=tf,
            sl=sl,
        )

        combined_trades_ce = pd.concat([combined_trades_ce, ce_trades], ignore_index=True)
        # print(combined_trades_ce.to_string())

        # print('Executing PE Trades')
        pe_trades = await trade_option_selling(
            df=pe_df,
            index=INDEX,
            strike=spot_atm,
            asset_class="P",
            expiry=current_expiry,
            signal_ma=signal_ma,
            no_of_candles=no_of_candles,
            tf=tf,
            sl=sl,
        )

        combined_trades_pe = pd.concat([combined_trades_pe, pe_trades], ignore_index=True)
        # print(combined_trades_pe.to_string())

    total_trades = pd.concat([total_trades, combined_trades_ce, combined_trades_pe], ignore_index=True)
    total_trades = total_trades.sort_values(by="Signal Generated At", ignore_index=True)
    return total_trades 

In [46]:
# weekly_data

In [48]:
def generate_stats(tb_expiry, signal_ma, no_of_candles, tf, sl):
    stats_df8 = pd.DataFrame(
        index=range(2022, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2022, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f'{signal_ma} , {no_of_candles}, {tf}, {sl}'
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [51]:
# SIMULATION

x = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
y = [1, 2, 3, 4, 5]
z = ["3m", "5m", "10m", "15m", "30m"]
sl = [1.25, 1.5, 1.75, 2]

stats_dictionary = {}
for i in x:
    for j in y:
        for k in z:
            for l in sl:
                print(f"Signal MA : {i} , No. of Candles : {j} , TimeFrame : {k} , SL {l}")
                tb = await trade(weekly_data, i, j, k, l)
                # print(len(tb))
                if len(tb) > 0:
                    tb_ce = tb[tb['Option Type'] == 'C']
                    tb_pe = tb[tb['Option Type'] == 'P']
                    stats = generate_stats(tb, i, j, k, l)
                    stats_ce = generate_stats(tb_ce, i, j, k, l)
                    stats_pe = generate_stats(tb_pe, i, j, k, l)
    
                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 8:
                            print("Overall Combined")
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df
    
                    # for overall_roi_dd_ratio, stats_df in stats_ce.items():
                    #     if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 8:
                    #         print("Only CE")
                    #         print(stats_df.to_string())
                    #         stats_dictionary[overall_roi_dd_ratio] = stats_df
    
                    # for overall_roi_dd_ratio, stats_df in stats_pe.items():
                    #     if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 8:
                    #         print("Only PE")
                    #         print(stats_df.to_string())
                    #         stats_dictionary[overall_roi_dd_ratio] = stats_df

Signal MA : 5 , No. of Candles : 1 , TimeFrame : 3m , SL 1.25
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 3m , SL 1.5
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 3m , SL 1.75
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 3m , SL 2
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 5m , SL 1.25
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 5m , SL 1.5
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 5m , SL 1.75
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 5m , SL 2
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 10m , SL 1.25
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 10m , SL 1.5
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 10m , SL 1.75
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 10m , SL 2
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 15m , SL 1.25
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 15m , SL 1.5
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 15m , SL 1.75
Signal MA : 5 , No. of Candles : 1 , TimeFrame : 15m , SL 2
Signal MA : 5 , 

/tmp/ipykernel_2852/888587993.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA : 5 , No. of Candles : 4 , TimeFrame : 30m , SL 2


/tmp/ipykernel_2852/888587993.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA : 5 , No. of Candles : 5 , TimeFrame : 3m , SL 1.25
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 3m , SL 1.5
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 3m , SL 1.75
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 3m , SL 2
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 5m , SL 1.25
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 5m , SL 1.5
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 5m , SL 1.75
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 5m , SL 2
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 10m , SL 1.25
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 10m , SL 1.5
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 10m , SL 1.75
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 10m , SL 2
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 15m , SL 1.25
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 15m , SL 1.5
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 15m , SL 1.75
Signal MA : 5 , No. of Candles : 5 , TimeFrame : 15m , SL 2
Signal MA : 5 , 

...

/tmp/ipykernel_2852/888587993.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA : 50 , No. of Candles : 5 , TimeFrame : 30m , SL 1.5
Signal MA : 50 , No. of Candles : 5 , TimeFrame : 30m , SL 1.75


/tmp/ipykernel_2852/888587993.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


Signal MA : 50 , No. of Candles : 5 , TimeFrame : 30m , SL 2


/tmp/ipykernel_2852/888587993.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
# # tb = await trade(weekly_data, i, j, k)
# # tb

In [ ]:
sorted_stats

In [24]:
tb = await trade(weekly_data, 20, 4, "30m", 2)
# tb

In [25]:
tb

Index  Strike Option Type      Expiry Signal Generated At Trade Type  \
0    nifty    8400           P  2017-01-25 2017-01-24 10:00:00       SELL   
1    nifty    9100           P  2017-03-30 2017-03-28 14:00:00       SELL   
2    nifty    9200           P  2017-04-27 2017-04-24 14:30:00       SELL   
3    nifty    9450           P  2017-05-25 2017-05-25 12:30:00       SELL   
4    nifty    9650           C  2017-06-29 2017-06-28 09:30:00       SELL   
5    nifty    9900           P  2017-07-27 2017-07-25 10:30:00       SELL   
6    nifty    9900           C  2017-08-31 2017-08-29 14:00:00       SELL   
7    nifty   10100           C  2017-09-28 2017-09-25 14:30:00       SELL   
8    nifty   10200           P  2017-10-26 2017-10-24 14:00:00       SELL   
9    nifty   10350           P  2017-11-30 2017-11-28 09:30:00       SELL   
10   nifty   10350           C  2017-11-30 2017-11-28 10:30:00       SELL   
11   nifty   10850           P  2018-01-25 2018-01-22 14:00:00       SELL   
12   nifty   10600           C  2018-02-22 2018-02-19 15:00:00       SELL   
13   nifty    9950           P  2018-03-28 2018-03-26 14:00:00       SELL   
14   nifty   10550           P  2018-04-26 2018-04-23 14:00:00       SELL   
15   nifty   10550           P  2018-05-31 2018-05-28 14:00:00       SELL   
16   nifty   10550           C  2018-05-31 2018-05-29 14:00:00       SELL   
17   nifty   10550           P  2018-05-31 2018-05-31 10:30:00       SELL   
18   nifty   10750           P  2018-06-28 2018-06-25 14:30:00       SELL   
19   nifty   10750           C  2018-06-28 2018-06-26 13:00:00       SELL   
20   nifty   10950           P  2018-07-26 2018-07-23 14:00:00       SELL   
21   nifty   11550           P  2018-08-30 2018-08-27 14:30:00       SELL   
22   nifty   11250           C  2018-09-27 2018-09-24 14:30:00       SELL   
23   nifty   10350           C  2018-10-25 2018-10-22 14:00:00       SELL   
24   nifty   10550           P  2018-11-29 2018-11-27 14:00:00       SELL   
25   nifty   10950           C  2018-12-27 2018-12-24 14:00:00       SELL   
26   nifty   10850           C  2019-01-31 2019-01-28 14:30:00       SELL   
27   nifty   11000           C  2019-02-14 2019-02-12 14:00:00       SELL   
28   nifty   10800           P  2019-02-28 2019-02-25 14:00:00       SELL   
29   nifty   10800           P  2019-02-28 2019-02-26 13:30:00       SELL   
30   nifty   10850           P  2019-03-07 2019-03-05 14:00:00       SELL   
31   nifty   11050           P  2019-03-14 2019-03-11 14:00:00       SELL   
32   nifty   11400           P  2019-03-20 2019-03-18 14:00:00       SELL   
33   nifty   11550           C  2019-03-28 2019-03-25 14:00:00       SELL   
34   nifty   11550           C  2019-03-28 2019-03-28 12:30:00       SELL   
35   nifty   11650           P  2019-04-04 2019-04-01 14:00:00       SELL   
36   nifty   11650           C  2019-04-04 2019-04-04 09:30:00       SELL   
37   nifty   11650           C  2019-04-11 2019-04-08 15:00:00       SELL   
38   nifty   11650           P  2019-04-11 2019-04-10 09:00:00       SELL   
39   nifty   11650           C  2019-04-11 2019-04-11 09:30:00       SELL   
40   nifty   11600           P  2019-04-18 2019-04-15 14:00:00       SELL   
41   nifty   11750           C  2019-04-25 2019-04-23 14:00:00       SELL   
42   nifty   11750           P  2019-04-25 2019-04-25 09:00:00       SELL   
43   nifty   11700           C  2019-05-09 2019-05-06 14:00:00       SELL   
44   nifty   11300           C  2019-05-16 2019-05-13 14:00:00       SELL   
45   nifty   11300           P  2019-05-16 2019-05-15 10:30:00       SELL   
46   nifty   11250           P  2019-05-23 2019-05-20 14:30:00       SELL   
47   nifty   11750           P  2019-05-30 2019-05-27 14:00:00       SELL   
48   nifty   12000           P  2019-06-06 2019-06-03 14:00:00       SELL   
49   nifty   12000           C  2019-06-06 2019-06-06 10:30:00       SELL   
50   nifty   11850           P  2019-06-13 2019-06-10 14:00:00       SELL   
5

In [26]:
tb_ce2 = tb[tb['Option Type'] == 'C']
tb_pe2 = tb[tb['Option Type'] == 'P']

In [27]:
stats2 = generate_stats(tb, 20, 4, "30m", 2)
roi_overall, stats_overall = next(iter(stats2.items()))
stats_overall

Total ROI Total Trades Win Rate Avg Profit% per Trade  \
2017      25.6660           11  81.8182                3.2973   
2018       9.3201           15  80.0000                1.9117   
2019      12.0952           62  61.2903                2.3414   
2020      93.7934           71  61.9718                5.0670   
2021      39.3035           77  58.4416                3.3935   
2022      24.1172           75  60.0000                3.0995   
2023      -3.3389           78  56.4103                1.5952   
2024     -17.4121           39  51.2821                2.6476   
Overall  183.5444          428  60.0467                3.0345   

        Avg Loss% per Trade Max Drawdown ROI/DD Ratio       Variation  
2017                -2.0050      -2.7393       9.3696  20 , 4, 30m, 2  
2018                -4.5402     -13.3506       0.6981  20 , 4, 30m, 2  
2019                -3.2032     -11.2979       1.0706  20 , 4, 30m, 2  
2020                -4.7834     -19.4108       4.8320  20 , 4, 30m, 2  
2021                -3.5439     -25.7440       1.5267  20 , 4, 30m, 2  
2022                -3.8454     -22.9254       1.0520  20 , 4, 30m, 2  
2023                -2.1625     -14.8247      -0.2252  20 , 4, 30m, 2  
2024                -3.7033     -27.3346      -0.6370  20 , 4, 30m, 2  
Overall             -3.4872     -33.0190       5.5587  20 , 4, 30m, 2

In [ ]:
# tb.to_csv('PSM nifty.csv')